# Soak Test Prototype

In [ ]:
test_case = "LVV-TXXXX"
test_exec = "LVV-EXXXX"

## Setup Environment

### Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import asyncio
import logging

import numpy as np

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS
from lsst.ts.observatory.control import RotType

from lsst.sitcom import vandv

### Get execution info 

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

### Confirm environment variables

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ.get("LSST_DDS_DOMAIN_ID", "Expected, not set."))

### Setup logging

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)
log = logging.getLogger("soak_test")
log.level = logging.DEBUG
log.addFilter(vandv.logger.filter_dds_read_queue_is_filling)
log.addFilter(vandv.logger.filter_dds_read_queue_is_full)

In [ ]:
script = salobj.Controller("Script", index=-20221110)
await asyncio.sleep(10) 

script.start_task

In [ ]:
domain = salobj.Domain()

script.log.addFilter(vandv.logger.filter_dds_read_queue_is_filling)
script.log.addFilter(vandv.logger.filter_dds_read_queue_is_full)

mtcs = MTCS(domain=domain, log=script.log)
mtcs.set_rem_loglevel(40)

await mtcs.start_task

In [ ]:
await mtcs.set_state(salobj.State.DISABLED, components=["mtmount"])

In [ ]:
# await mtcs.disable_ccw_following()

In [ ]:
az = mtcs.rem.mtmount.tel_azimuth.get().actualPosition
el = mtcs.rem.mtmount.tel_elevation.get().actualPosition

print(az, el)

In [ ]:
await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=2, elevation=62)

In [ ]:
await mtcs.rem.mtmount.cmd_homeBothAxes.start(timeout=300)
# await mtcs.disable_ccw_following()

## Helper Functions

In [ ]:
# Simulate current position while we don't test with the hardware
# _az = 0
# _el = 80

In [ ]:
def random_walk_azel(n_steps, 
                     radius=3.5, 
                     min_az=-200., 
                     max_az=+200, 
                     min_el=30, 
                     max_el=75,
                     verbose=True,
                    ):

    if verbose:
        print(f"{'Steps':>10s}{'New Az':>10s}{'New El':>10s}")
        
    step = 0
    while step < n_steps:
        
        current_az = mtcs.rem.mtmount.tel_azimuth.get()
        current_az = current_az.actualPosition
        # current_az = _az
        offset_az = np.sqrt(radius) * (2 * np.random.rand() - 1)
        new_az = current_az + offset_az
                
        current_el = mtcs.rem.mtmount.tel_elevation.get()
        current_el = current_el.actualPosition
        # current_el = _el
        offset_el = np.sqrt(radius) * (2 * np.random.rand() - 1)
        new_el = current_el + offset_el
        
        if new_az <= min_az or new_az >= max_az:
            new_az = current_az - offset_az
            
        if new_el <= min_el or new_el >= max_el:
            new_el = current_el - offset_el

        if verbose:
            print(f"{step:10d}{new_az:10.2f}{new_el:10.2f}")

        yield new_az, new_el
        step += 1

In [ ]:
await mtcs.move_rotator(1)

In [ ]:
await mtcs.disable_ccw_following()

In [ ]:
script.log.info("Soak Test - START")
for az, el in random_walk_azel(100):
    # _az, _el = az, el
    await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=az, elevation=el, timeout=300)
    await asyncio.sleep(5)

script.log.info("Soak Test - DONE")

In [ ]:
mtcs.check.mtm2 = False
mtcs.check.mtm1m3 = False
mtcs.check.mthexapod_1 = False
mtcs.check.mthexapod_2 = False
mtcs.check.mtaos = False
mtcs.check.mtdome = False
mtcs.check.mtdometrajectory = False

# await mtcs.disable_ccw_following()

In [ ]:
az = mtcs.rem.mtmount.tel_azimuth.get().actualPosition
el = mtcs.rem.mtmount.tel_elevation.get().actualPosition

print(az, el)

In [ ]:
from astropy.time import Time

In [ ]:
print(Time.now())

In [ ]:
script.log.info("Slew and Track - START")

radec = mtcs.radec_from_azel(5, 65)
await mtcs.slew_icrs(radec.ra, radec.dec, rot=0, rot_type=RotType.PhysicalSky) 

script.log.info("Slew and Track - DONE")

In [ ]:
print(Time.now())

In [ ]:
await mtcs.stop_tracking()

In [ ]:
task.cancel()

In [ ]:
task.done()

In [ ]:
21:25 - 21:32

In [ ]:
# task = asyncio.create_task(
#     mtcs.slew_icrs(radec.ra, radec.dec, rot=0, rot_type=RotType.Physical)
# ) 

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtmount"])